In [1]:
import pandas as pd
import re
import os
import sys
import pickle as pkl
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import json

df_annotations = pd.read_csv("all_annotations.csv")


# Get data

In [2]:
# with open("protocols.pkl.gz", "rb") as f:
#     file_to_pages = pkl.load(f)

In [3]:
# import spacy
# nlp = spacy.blank("en")

In [4]:
# texts = []

# for i in range(len(df_annotations)):
#     file_name = df_annotations.file.iloc[i]
    
#     pages= file_to_pages[file_name]
    
#     text = ""
#     for page in pages:
#         doc = nlp(page)
        
#         is_include = [False] * len(doc)
#         for tok in doc:
#             if tok.text.lower() in {"phase"}:
#                 for token_index in range(tok.i - 20, tok.i + 20):
#                     if token_index >= 0 and token_index < len(doc):
#                         is_include[token_index] = True
        
#         for token_index in range(len(doc)):
#             if is_include[token_index]:
#                 text += doc[token_index].text + doc[token_index].whitespace_
                
                
#     if text == "":
#         print ("nothing found", file_name, df_annotations.phase.iloc[i])
#         text = " ".join(pages)
#         if len(text) > 100000:
#             text = text[:100000]
#     else:
#         print ("found", file_name, df_annotations.phase.iloc[i])

#     texts.append(text)
# df_annotations["text"] = texts

In [5]:
# df_annotations.to_csv("filtered_for_phase.csv.bz2")

In [6]:
df_annotations = pd.read_csv("filtered_for_phase.csv.bz2")

In [7]:
# del file_to_pages

In [8]:
df_annotations_phase_known = df_annotations[~df_annotations.phase.isna()]

In [9]:
len(df_annotations_phase_known), len(df_annotations)

(11591, 11924)

In [10]:
#df_annotations_num_arms_known = df_annotations_num_arms_known.sample(1000)

In [11]:
df_annotations_phase_known.phase.value_counts()

Phase 2            4317
Phase 3            2725
Phase 4            1461
Phase 1            1154
Phase 1/Phase 2     983
Not Applicable      519
Phase 2/Phase 3     297
Early Phase 1       135
Name: phase, dtype: int64

In [12]:
phase_map = {"Phase 2":"2",
"Phase 3":"3",
"Phase 4":"4",
"Phase 1":"1",
"Phase 1/Phase 2":"1.5",
"Not Applicable":"0",
"Phase 2/Phase 3":"2.5",
"Early Phase 1":"0.5"}
df_annotations_phase_known["phase_clean"] = df_annotations_phase_known["phase"].map(phase_map)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
len(phase_map)

8

In [14]:
df_train = df_annotations_phase_known[df_annotations_phase_known.train_val == "train"]

In [15]:
df_val = df_annotations_phase_known[df_annotations_phase_known.train_val == "val"]

In [16]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score, auc, roc_curve
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.pipeline import make_pipeline

NUM_FEATURES = 1500


stops = set(stopwords.words('english')).union(set(stopwords.words('french')))
stops.remove("both")
stops.remove("all")

# Specially engineered regex to include 95%, 95%ci, etc
vectoriser = CountVectorizer(lowercase=True,
                         stop_words = stops, max_features= NUM_FEATURES,
                             ngram_range=[1,2]
                            )
transformer = TfidfTransformer()

nb = MultinomialNB()
nb = ComplementNB()
model = make_pipeline(vectoriser, transformer, nb)

In [17]:
model.fit(df_train.text, df_train.phase_clean)

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_features=1500, ngram_range=[1, 2],
                                 stop_words={'a', 'about', 'above', 'after',
                                             'again', 'against', 'ai', 'aie',
                                             'aient', 'aies', 'ain', 'ait',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'au',
                                             'aura', 'aurai', 'auraient',
                                             'aurais', 'aurait', 'auras',
                                             'aurez', 'auriez', ...})),
                ('tfidftransformer', TfidfTransformer()),
                ('complementnb', ComplementNB())])

In [18]:
df_val["y_pred"]  = model.predict(df_val.text)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [19]:
phase_ground_truths = df_val.phase_clean
pred_phase = df_val["y_pred"]

In [20]:
acc = accuracy_score(phase_ground_truths, pred_phase)
print (f"Phase accuracy {acc}")

Phase accuracy 0.5099337748344371


In [24]:
phase_ground_truths.value_counts()

2      395
3      249
4      126
1      108
1.5     89
0       46
2.5     31
0.5     13
Name: phase_clean, dtype: int64

In [30]:
labels = list(sorted(phase_map.values()))
from sklearn.metrics import confusion_matrix
print (labels)
confusion_matrix(phase_ground_truths, pred_phase, labels=labels)


['0', '0.5', '1', '1.5', '2', '2.5', '3', '4']


array([[  0,   0,   0,   2,   8,   0,   2,  34],
       [  0,   0,   0,   1,   3,   0,   1,   8],
       [  0,   0,  48,  12,  19,   0,  17,  12],
       [  1,   0,   5,  36,  25,   0,   4,  18],
       [  3,   0,  12,   7, 209,   0, 112,  52],
       [  0,   0,   0,   0,   4,   0,  14,  13],
       [  2,   0,   6,   3,  37,   0, 167,  34],
       [  0,   0,   6,   0,   5,   0,  36,  79]])

In [21]:
ConfusionMatrixDisplay.from_predictions(phase_ground_truths, pred_phase)
plt.xticks(rotation=90)
;

AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_predictions'